### 1) Write a Python script that fetches holiday data and saves it as a csv file named "holidays.csv".​ To obtain the data, you can either screen scrape the Time and Date​ site, or parse the API (free trial​ account required).

In [2]:
import pandas as pd
import numpy as np
import requests
import lxml.html
import datetime

In [3]:
page = requests.get("https://www.timeanddate.com/holidays/us/")

In [4]:
tree = lxml.html.fromstring(page.text)

In [5]:
dates = tree.xpath("//th/text()")[5:]
len(dates)

402

In [6]:
days = tree.xpath("//tr/td[1]/text()")[:402]
len(days)

402

In [7]:
holiday_names = tree.xpath("//td/a/text()")
len(holiday_names)

402

In [8]:
holiday_type = tree.xpath("//tr/td[3]/text()")
len(holiday_type)

402

In [9]:
t = tree.xpath("//tr/td[4]")
details = []
for elem in t:
    val = elem.xpath('text()')
    temp = ''
    temp1 = ''
    if(len(val)>1):
        if(val[0]=='All except '):
            temp1 = temp1 + 'All except '
            
            for x in range(1,len(val)+1):
                tt = elem.xpath("span["+str(x)+"]/text()")
                temp1 = temp1 + tt[0] + ','
            details.append([temp1])
        else:
            ff = elem.xpath("span/text()")
            for x in ff:
                temp = temp+x + ','
            details.append([temp])       
    else:
        details.append(val)
len(details)

402

In [10]:
holidays_df = pd.DataFrame({'Date': dates,
                            'Day': days,
                            'Name':holiday_names,
                            'Type':holiday_type,
                            'Details': details})

In [11]:
holidays_df['Details'] = holidays_df['Details'].apply(lambda x: ','.join(map(str, x)))

In [12]:
holidays_df.head()

,Date,Day,Name,Type,Details
0,Jan 1,Tuesday,New Year's Day,Federal Holiday,
1,Jan 4,Friday,World Braille Day,Worldwide observance,
2,Jan 6,Sunday,Epiphany,Christian,
3,Jan 7,Monday,Orthodox Christmas Day,Orthodox,
4,Jan 7,Monday,International Programmers' Day,Worldwide observance,


In [13]:
holidays_df.to_csv("holidays.csv")

### 2) Using the csv file from 1), write a​ Python​ function that returns a list of dictionaries of the next 10 upcoming U.S. holidays and dates. 

E.g.


def holidays​():​

    pass



holidays​()​

[
    {
        "name​": "Bastille Day​",
        "date": "Sunday, July 14, 2019",
        "type": "Observance",
        "details​": ""
    },
    ...
]



In [49]:
holidays = pd.read_csv("holidays.csv")
holidays.head(10)

,Unnamed: 0,Date,Day,Name,Type,Details
0,0,Jan 1,Tuesday,New Year's Day,Federal Holiday,
1,1,Jan 4,Friday,World Braille Day,Worldwide observance,
2,2,Jan 6,Sunday,Epiphany,Christian,
3,3,Jan 7,Monday,Orthodox Christmas Day,Orthodox,
4,4,Jan 7,Monday,International Programmers' Day,Worldwide observance,
5,5,Jan 13,Sunday,Stephen Foster Memorial Day,Observance,
6,6,Jan 14,Monday,Orthodox New Year,Orthodox,
7,7,Jan 18,Friday,Lee-Jackson Day,State holiday,Virginia
8,8,Jan 19,Saturday,Robert E. Lee's Birthday,State holiday,Florida
9,9,Jan 19,Saturday,Confederate Heroes' Day,State holiday,Texas


In [50]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 6 columns):
Unnamed: 0    402 non-null int64
Date          402 non-null object
Day           402 non-null object
Name          402 non-null object
Type          402 non-null object
Details       395 non-null object
dtypes: int64(1), object(5)
memory usage: 18.9+ KB


In [51]:
#remove the - in the Date Column
holidays["Date"] = holidays["Date"].str.replace("-"," ")

In [52]:
#Adding a Year column
holidays["Year"] = 2019

In [53]:
#Splitting the Date column into Month & Day
new = holidays["Date"].str.split(" ", expand = True)
holidays["Month"]= new[0] 
holidays["Dates"]= new[1]
holidays.drop(columns =["Date"], inplace = True)

In [54]:
#Converting the Month,Date,Year to One column and converting to Datetime object
holidays['FullDate'] = holidays['Dates'].map(str) + '-' + holidays['Month'].map(str) + '-' + holidays['Year'].map(str)
holidays['FullDate']= pd.to_datetime(holidays['FullDate'])
holidays.head()

,Unnamed: 0,Day,Name,Type,Details,Year,Month,Dates,FullDate
0,0,Tuesday,New Year's Day,Federal Holiday,,2019,Jan,1,2019-01-01
1,1,Friday,World Braille Day,Worldwide observance,,2019,Jan,4,2019-01-04
2,2,Sunday,Epiphany,Christian,,2019,Jan,6,2019-01-06
3,3,Monday,Orthodox Christmas Day,Orthodox,,2019,Jan,7,2019-01-07
4,4,Monday,International Programmers' Day,Worldwide observance,,2019,Jan,7,2019-01-07


In [20]:
#Converting Dataframe to a list of dictionaries based on the records
dict_holidays = holidays.to_dict('records')
dict_holidays[:5]

[{'Unnamed: 0': 0,
  'Day': 'Tuesday',
  'Name': "New Year's Day",
  'Type': 'Federal Holiday',
  'Details': '\xa0',
  'Year': 2019,
  'Dates': 'Jan',
  'Month': '1',
  'FullDate': Timestamp('2019-01-01 00:00:00')},
 {'Unnamed: 0': 1,
  'Day': 'Friday',
  'Name': 'World Braille Day',
  'Type': 'Worldwide observance',
  'Details': '\xa0',
  'Year': 2019,
  'Dates': 'Jan',
  'Month': '4',
  'FullDate': Timestamp('2019-01-04 00:00:00')},
 {'Unnamed: 0': 2,
  'Day': 'Sunday',
  'Name': 'Epiphany',
  'Type': 'Christian',
  'Details': '\xa0',
  'Year': 2019,
  'Dates': 'Jan',
  'Month': '6',
  'FullDate': Timestamp('2019-01-06 00:00:00')},
 {'Unnamed: 0': 3,
  'Day': 'Monday',
  'Name': 'Orthodox Christmas Day',
  'Type': 'Orthodox',
  'Details': '\xa0',
  'Year': 2019,
  'Dates': 'Jan',
  'Month': '7',
  'FullDate': Timestamp('2019-01-07 00:00:00')},
 {'Unnamed: 0': 4,
  'Day': 'Monday',
  'Name': "International Programmers' Day",
  'Type': 'Worldwide observance',
  'Details': '\xa0',
  'Ye

In [21]:
def holiday_10():
    hols_list = []
    curr_date = datetime.datetime.today()
    print("List Of Holidays:\n")
    for hols in dict_holidays:
        if(curr_date <= hols['FullDate']):
            hols_list.append(hols)
              
    sorted_hols = sorted(hols_list, key=lambda x: x['FullDate'])
    print("\n")
    print(sorted_hols[:10]) 
        
holiday_10()

List Of Holidays:



[{'Unnamed: 0': 228, 'Day': 'Thursday', 'Name': 'Nelson Mandela Day', 'Type': 'United Nations observance', 'Details': '\xa0', 'Year': 2019, 'Dates': 'Jul', 'Month': '18', 'FullDate': Timestamp('2019-07-18 00:00:00')}, {'Unnamed: 0': 229, 'Day': 'Wednesday', 'Name': 'Pioneer Day', 'Type': 'State holiday', 'Details': 'Utah', 'Year': 2019, 'Dates': 'Jul', 'Month': '24', 'FullDate': Timestamp('2019-07-24 00:00:00')}, {'Unnamed: 0': 230, 'Day': 'Saturday', 'Name': 'National Korean War Veterans Armistice Day', 'Type': 'Observance', 'Details': '\xa0', 'Year': 2019, 'Dates': 'Jul', 'Month': '27', 'FullDate': Timestamp('2019-07-27 00:00:00')}, {'Unnamed: 0': 231, 'Day': 'Sunday', 'Name': 'World Hepatitis Day', 'Type': 'United Nations observance', 'Details': '\xa0', 'Year': 2019, 'Dates': 'Jul', 'Month': '28', 'FullDate': Timestamp('2019-07-28 00:00:00')}, {'Unnamed: 0': 232, 'Day': 'Sunday', 'Name': "Parents' Day", 'Type': 'Observance', 'Details': '\xa0', 'Year': 2019, 'Dat

### 3) Add a keyword argument "holidayType​"​ to the function from 2) such that it only returns holidays matching the given type, 

e.g., 


def holidays​(holidayType​​=None​):​

    pass

holidays​​(holidayType​​​="federal")    

[
   {
       "name​": "Labor​ Day​",
       "date": "Monday​, September 2, 2019",
       "type": "Federal Holiday​",
       "details​": ""
   },
   ...
]

In [47]:
def holiday_type_func(holiday_type=None):
    hols_list = []
    curr_date = datetime.datetime.today()
    print("List Of Holidays:\n")
    for hols in dict_holidays:
        if(curr_date <= hols['FullDate']):
            if(hols['Type'] == holiday_type):
                hols_list.append(hols)
            elif(hols['Type'] == holiday_type):
                hols_list.append(hols)
              
    sorted_hols = sorted(hols_list, key=lambda x: x['FullDate'])
    print("\n")
    print(sorted_hols) 
    return

holiday_type_func("Federal Holiday")

List Of Holidays:



[{'Unnamed: 0': 259, 'Day': 'Monday', 'Name': 'Labor Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 2019, 'Dates': 'Sep', 'Month': '2', 'FullDate': Timestamp('2019-09-02 00:00:00')}, {'Unnamed: 0': 312, 'Day': 'Monday', 'Name': 'Columbus Day', 'Type': 'Federal Holiday', 'Details': 'All except AK,AR,CA,DE,FL,HI,MI,MN,ND,NM,NV,OR,SD,TX,VT,WA,WY,', 'Year': 2019, 'Dates': 'Oct', 'Month': '14', 'FullDate': Timestamp('2019-10-14 00:00:00')}, {'Unnamed: 0': 343, 'Day': 'Monday', 'Name': 'Veterans Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 2019, 'Dates': 'Nov', 'Month': '11', 'FullDate': Timestamp('2019-11-11 00:00:00')}, {'Unnamed: 0': 356, 'Day': 'Thursday', 'Name': 'Thanksgiving Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 2019, 'Dates': 'Nov', 'Month': '28', 'FullDate': Timestamp('2019-11-28 00:00:00')}, {'Unnamed: 0': 396, 'Day': 'Wednesday', 'Name': 'Christmas Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 201